In [54]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

In [55]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")


In [56]:
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,4a16050093a99092,"No, you have to prove that I can't.",0,0,0,0,0,0
1,6bc2122dc27e275a,Re: My major edit summaries \n\nI have receive...,0,0,0,0,0,0
2,5570f7207bade92f,I depersonalized it. It's sad how you don't ob...,0,0,0,0,0,0
3,a17d6408ffb98139,Thats because I was blocked before I could dis...,0,0,0,0,0,0
4,11b1d25d77ffd834,"I apologise for this, I was just angry with ha...",0,0,0,0,0,0


In [57]:
test_df.head()

,id,comment_text
0,f88f17ce73d55ad9,"Mark Teixiera\nPer WP:CRYSTAL, we only change ..."
1,6a9f4af6d51fc9e2,"""\n\n Reblock \n\nReblock that IP you unblocke..."
2,c44f6d09834d1058,"""::::Censorship isn't an answer. Some things s..."
3,b1b2e8a5731c75d0,This is more fun than acrually editting!! \n\n...
4,f25b7274d05d8290,SarekOfVulcan RFA \n\nThank you for !voting on...


In [58]:
train_df.isnull().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

### Remove Punctuation

In [59]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [60]:
tqdm.pandas()
train_df['text_clean'] = train_df['comment_text'].progress_apply(lambda x : "".join([c for c in x if c not in string.punctuation])) 

/home/saikiran/.local/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [61]:
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,text_clean
0,4a16050093a99092,"No, you have to prove that I can't.",0,0,0,0,0,0,No you have to prove that I cant
1,6bc2122dc27e275a,Re: My major edit summaries \n\nI have receive...,0,0,0,0,0,0,Re My major edit summaries \n\nI have received...
2,5570f7207bade92f,I depersonalized it. It's sad how you don't ob...,0,0,0,0,0,0,I depersonalized it Its sad how you dont objec...
3,a17d6408ffb98139,Thats because I was blocked before I could dis...,0,0,0,0,0,0,Thats because I was blocked before I could dis...
4,11b1d25d77ffd834,"I apologise for this, I was just angry with ha...",0,0,0,0,0,0,I apologise for this I was just angry with hav...


In [62]:
train_df['text_clean'][1]

'Re My major edit summaries \n\nI have received your message As I vote for or against candidates for administrators I consider edit summary usage important What exactly are WikiSin and mathbot'

In [63]:
train_df['text_clean'] = train_df['text_clean'].apply(lambda x: "".join(x.splitlines()))

In [64]:
train_df['text_clean'][1]

'Re My major edit summaries I have received your message As I vote for or against candidates for administrators I consider edit summary usage important What exactly are WikiSin and mathbot'

In [65]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer


In [66]:
tokenizer = RegexpTokenizer(r'\w+')
train_df['token']= train_df['text_clean'].apply(lambda x : tokenizer.tokenize(x.lower())) 


In [67]:
#train_df['token'] = train_df['token'].progress_apply(lambda x : [w for w in x if w not in set(stopwords.words('english'))])

In [68]:
train_df['token'] = train_df['token'].progress_apply(lambda x : [WordNetLemmatizer().lemmatize(i) for i in x])

In [69]:
train_df['token']

0                 [no, you, have, to, prove, that, i, cant]
1         [re, my, major, edit, summary, i, have, receiv...
2         [i, depersonalized, it, it, sad, how, you, don...
3         [thats, because, i, wa, blocked, before, i, co...
4         [i, apologise, for, this, i, wa, just, angry, ...
                                ...                        
111694    [honestly, olaf, the, pov, fork, argument, jus...
111695    [public, domain, image, needed, can, anyone, u...
111696    [unban, this, ip, address, or, a, new, online,...
111697    [rv, sorry, about, the, rv, went, to, check, o...
111698    [since, the, name, of, the, mountain, wa, not,...
Name: token, Length: 111699, dtype: object

In [70]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re
def clean_text(text):
    """
      - Remove Punctuations
      - Tokenization
      - Remove Stopwords
      - stemming/lemmatizing
    """
    stops = set(stopwords.words("english"))
    wl = nltk.WordNetLemmatizer()
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    tokens = re.split("\W+", text)
    text = [word for word in tokens if word not in stops]
    text = "".join(text.splitlines())
    text = [wl.lemmatize(word) for word in text]
    return text


In [71]:
tfidvect = TfidfVectorizer()
textvector = tfidvect.fit_transform(train_df['comment_text'])
print(textvector.shape)
#print(tfidvect.get_feature_names())

(111699, 152803)


In [72]:
testvector = tfidvect.transform(test_df['comment_text'])
print(testvector.shape)

(47872, 152803)


In [73]:
output_labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
x= textvector
x_test = testvector

In [74]:
from sklearn.linear_model import LogisticRegression


In [75]:
preds = np.zeros((len(test_df), len(output_labels)))

def get_model(y):
    model = LogisticRegression(C=4, dual=True)
    return model.fit(x,y.values)

for i, j in enumerate(output_labels):
    print('fit', j)
    m = get_model(train_df[j])
    preds[:,i] = m.predict_proba(x_test)[:,1]

fit toxic


/home/saikiran/.local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


fit severe_toxic
fit obscene
fit threat
fit insult
fit identity_hate


In [1]:
subm= pd.read_csv('sample_submission.csv')
submid = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)
submission.to_csv('submission.csv', index=False)

NameError: name 'pd' is not defined